# Plots interaction

## Step1 : Plot of a simple image
using Plot2D

In [1]:
%gui qt

### Step 1.1 load data from data/lena.hdf5 
as a numpy array of float32 (numpy.array(data, dtype='float32'))

In [20]:
# input using .hdf5
import h5py
import numpy
dataPath='data/lena.hdf5'
f=h5py.File(dataPath)
from silx.io.utils import h5ls
h5ls(dataPath)

'<HDF5 dataset "lena": shape (512, 512), type "<i8">\n'

In [21]:
image=numpy.array(f['lena'], dtype='float32')

### Step 1.2 plot the image
![simple-image](img/plot_lena.png)

In [22]:
from silx.gui.plot import Plot2D
plotImage=Plot2D()
plotImage.addImage(image, origin=(0, 0), legend='lena')  # Plot the RGB image instead of the previous data
plotImage.show()

## Step 2 : plot of the intensity distribution

### Step 2.1 create the histogramnd 
using silx.math.histogram.Histogramnd

In [23]:
from silx.math import histogram
histo, w_histo, edges = histogram.Histogramnd(image.flatten(), n_bins=256, histo_range=[0,256])

### Step 2.2 create the corresponding plot
using silx.gui.plot.Plot1d

In [24]:
from silx.gui.plot import Plot1D
plotHisto = Plot1D()
plotHisto.addCurve(range(256), histo, legend='intensity')
plotHisto.show()

## Step 3 : create a QPlot action wich create and pop up the curve of the intensity distribution in the pixel
silx.gui.plot.PlotActions.PlotAction
![simple-image](img/plot_image-pixel_intensity.png)

In [25]:
from silx.gui.plot.PlotActions import PlotAction 
class ComputeHistogramAction(PlotAction):
    """QAction to cimpute the intensity distribution on the current image

    :param plot: :class:`.PlotWidget` instance on which to operate
    :param parent: See :class:`QAction`
    """
    def __init__(self, plot, parent=None):
        PlotAction.__init__(self,
                            plot,
                            icon='shape-circle',
                            text='Compute image intensity distribution',
                            tooltip='Compute image intensity distribution',
                            triggered=self.computeIntensityDistribution,
                            parent=parent)

    def computeIntensityDistribution(self):
        """Get the active image and compute the image intensity distribution"""
        # By inheriting from PlotAction, we get access to attribute self.plot
        # which is a reference to the PlotWindow
        activeImage = self.plot.getActiveImage()

        if activeImage is not None:
            from silx.math import histogram
            histo, w_histo, edges = histogram.Histogramnd(activeImage[0].flatten(), n_bins=256, histo_range=[0,256])
            
            from silx.gui.plot import Plot1D
            plotHistogram = Plot1D()
            plotHistogram.addCurve(range(256), histo, legend='pixel intensity')
            plotHistogram.show()

In [26]:
plotHisto.clear()
myaction=ComputeHistogramAction(plotImage)
toolBar=plotImage.toolBar()
toolBar.addAction(myaction)
plotImage.show()

# Step 4 : automatically show the pixel intensity histogram when the image change
using plotImage.sigActiveImageChanged.connect(plotHisto)

In [27]:
def computeIntensityDistribution():
        """Get the active image and compute the image intensity distribution"""
        # By inheriting from PlotAction, we get access to attribute self.plot
        # which is a reference to the PlotWindow
        activeImage = plotImage.getActiveImage()

        if activeImage is not None:
            from silx.math import histogram
            histo, w_histo, edges = histogram.Histogramnd(activeImage[0].flatten(), n_bins=256, histo_range=[0,256])
            
            from silx.gui.plot import Plot1D
            plotHistogram = Plot1D()
            plotHistogram.addCurve(range(256), histo, legend='pixel intensity')
            plotHistogram.show()

In [28]:
plotImage=Plot2D()
plotImage.sigActiveImageChanged.connect(computeIntensityDistribution)
plotImage.addImage(image, origin=(0, 0), legend='lena')  # Plot the RGB image instead of the previous data

'lena'